# Upload demographic data to database
Natalia Vélez, April 2021

In [1]:
import pandas as pd
import pymongo

Connect to database:

In [2]:
# Connect:
keyfile = '../6_database/credentials.key'
creds = open(keyfile, "r").read().splitlines()
myclient = pymongo.MongoClient('134.76.24.75', username=creds[0], password=creds[1], authSource='ohol') 
db = myclient.ohol

print(db)
print(db.list_collection_names())

Database(MongoClient(host=['134.76.24.75:27017'], document_class=dict, tz_aware=False, connect=True, authsource='ohol'), u'ohol')
[u'maplogs', u'tech_tree', u'objects', u'expanded_transitions', u'transitions', u'activity_matrix.files', u'activity_matrix.chunks', u'activity_labels', u'categories']


Load lifelogs:

In [3]:
life_df = pd.read_csv('outputs/all_lifelogs_compact.tsv', sep='\t', index_col=0)
life_df.head()

/n/home15/nvelezalicea/.conda/envs/ohol/lib/python2.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,player,avatar,tBirth,parent,birth,tDeath,death,age,cause_of_death,birthX,birthY,deathX,deathY,first,last
0,5ab4f808b01db8bab564fa726ca7dd0439205d4a,4108849,1617678319,4108781,[-13743 -77],1.617679e+09,[-13797 -104],3.59,hunger,-13743,-77,-13797.0,-104.0,TED,GFELL
1,5ab4f808b01db8bab564fa726ca7dd0439205d4a,4108848,1617678300,4108786,[-15498 362],1.617678e+09,[-15498 362],0.22,disconnect,-15498,362,-15498.0,362.0,HERA,POLLY
2,dadea1a71832326c598df59059cf42102e979707,4108842,1617678238,4108778,[-15498 362],1.617680e+09,[-15502 363],22.16,murdered,-15498,362,-15502.0,363.0,SPARTAN,POLLY
3,328dc412f542693dc20d084e99a7367e4fe4aae4,4108839,1617678208,4108784,[-13754 -85],1.617682e+09,[-13765 -64],60.00,oldAge,-13754,-85,-13765.0,-64.0,SERANA,GFELL
4,e65b7bfa037a3287869cb682b648c68e52ad05d4,4108837,1617678173,4108782,[-15796 -205],1.617679e+09,[-15803 -204],14.85,murdered,-15796,-205,-15803.0,-204.0,TJ,NaN


Load family registry:

In [4]:
family_df = pd.read_csv('outputs/family_playerID.tsv', sep='\t', index_col = 0)
print(family_df.shape)
family_df.head()

(1830190, 2)


,avatar,family
0,4108692,time-1617674434_eve-4108692_name-NOTIS
1,4108622,time-1617672864_eve-4108622_name-JONES
2,4108226,time-1617661499_eve-4108226_name-SAD
3,4108225,time-1617661497_eve-4108225_name-LOSTAUNAU
4,4108220,time-1617661402_eve-4108220_name-AA


Compute expertise:

In [5]:
expert_df = life_df[['avatar', 'player', 'tBirth', 'age']]
expert_df = expert_df.sort_values(by=['player', 'tBirth'])
expert_df['n_lives'] = expert_df.groupby(['player']).cumcount()
expert_df['gametime'] = expert_df.groupby(['player'])['age'].transform(lambda x: x.cumsum().shift())
expert_df['gametime'] = expert_df['gametime'].fillna(0)
expert_df = expert_df.drop(columns=['tBirth', 'age']).reset_index(drop=True)

expert_df.head(10)

,avatar,player,n_lives,gametime
0,630641,00006debdd4d630a4a5aa395135a70a69e0db01e,0,0.00
1,630737,00006debdd4d630a4a5aa395135a70a69e0db01e,1,13.88
2,630799,00006debdd4d630a4a5aa395135a70a69e0db01e,2,18.86
3,214206,0001394319e5e63bb74219fef647e9ff24507a21,0,0.00
4,214530,0001394319e5e63bb74219fef647e9ff24507a21,1,33.78
5,214960,0001394319e5e63bb74219fef647e9ff24507a21,2,41.03
6,215019,0001394319e5e63bb74219fef647e9ff24507a21,3,47.76
7,2783339,0002e5ea5ce7cfd761135d255a245a3344af4377,0,0.00
8,2783430,0002e5ea5ce7cfd761135d255a245a3344af4377,1,60.00
9,2784552,0002e5ea5ce7cfd761135d255a245a3344af4377,2,83.77


Merge:

In [6]:
merged_data = pd.merge(life_df, family_df, on='avatar')
merged_data = pd.merge(merged_data, expert_df, on='avatar')

print('Before merge: %s' % str(life_df.shape))
print('After merge: %s' % str(merged_data.shape))
merged_data = merged_data.drop(columns=['birth','death'])
merged_data = merged_data.to_dict('records')
print(merged_data[0])

Before merge: (2226610, 15)
After merge: (1830190, 19)
{'cause_of_death': 'hunger', 'last': 'GFELL', 'birthY': -77, 'parent': 4108781, 'birthX': -13743, 'gametime': 3130.2700000000004, 'age': 3.59, 'deathX': -13797.0, 'deathY': -104.0, 'tBirth': 1617678319, 'avatar': 4108849, 'family': 'time-1617611732_eve-4106678_name-GFELL', 'first': 'TED', 'n_lives': 142, 'tDeath': 1617678535.0, 'player_x': '5ab4f808b01db8bab564fa726ca7dd0439205d4a', 'player_y': '5ab4f808b01db8bab564fa726ca7dd0439205d4a'}


Upload merged dataframe to database:

In [8]:
life_col = db.lifelogs
life_col.insert_many(merged_data)